In [1]:
def load_doc(filename):
    file  = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [2]:
raw_text = load_doc('rhyme.txt')
print(raw_text)

Sing a song of sixpence,
A pocket full of rye.
Four and twenty blackbirds,
Baked in a pie.

When the pie was opened
The birds began to sing;
Wasn ' t that a dainty dish,
To set before the king.

The king was in his counting house,
Counting out his money;
The queen was in the parlour,
Eating bread and honey.

The maid was in the garden,
Hanging out the clothes,
When down came a blackbird
And pecked off her nose.



In [3]:
tokens = raw_text.split()
raw_text = ' '.join(tokens)

In [4]:
# Create sequences

In [5]:
length = 10
sequences = list()
for i in range(length, len(raw_text)):
    seq = raw_text[i-length:i+1]
    sequences.append(seq)
print("Total Sequences: %d"%len(sequences))

Total Sequences: 401


In [6]:
# Save sequences

In [7]:
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [8]:
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

In [9]:
# Load data

In [10]:
in_filename = "char_sequences.txt"
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

In [11]:
# Encode sequences

In [12]:
chars = sorted(list(set(raw_text)))
mapping = dict((c,i) for i,c in enumerate(chars))

In [13]:
sequences = list()
for line in lines:
    encoded_seq = [mapping[char] for char in line]
    sequences.append(encoded_seq)

In [14]:
vocab_size = len(mapping)
print('Vocabulary Size: %d'%vocab_size)

Vocabulary Size: 38


In [17]:
from numpy import array
sequences = array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

In [18]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [19]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [21]:
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
def define_model(X):
    model = Sequential()
    model.add(LSTM(75, input_shape=(X.shape[1],X.shape[2])))
    model.add(Dense(vocab_size, activation="softmax"))
    model.compile(loss="categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    model.summary()
    plot_model(model, to_file="model.png", show_shapes=True)
    return model

In [22]:
model = define_model(X)

W0505 10:48:48.102274 139830229423936 deprecation.py:506] From /home/arvind/.local/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1633: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 75)                34200     
_________________________________________________________________
dense_1 (Dense)              (None, 38)                2888      
Total params: 37,088
Trainable params: 37,088
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(X,y,epochs=100, verbose=2)

W0505 10:49:42.421247 139830229423936 deprecation.py:323] From /home/arvind/.local/lib/python3.6/site-packages/keras/optimizers.py:550: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.
W0505 10:49:42.635986 139830229423936 module_wrapper.py:136] From /home/arvind/.local/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/100
 - 1s - loss: 3.6124 - accuracy: 0.0823
Epoch 2/100
 - 0s - loss: 3.4857 - accuracy: 0.1920
Epoch 3/100
 - 0s - loss: 3.1723 - accuracy: 0.1945
Epoch 4/100
 - 0s - loss: 3.0322 - accuracy: 0.1945
Epoch 5/100
 - 0s - loss: 2.9988 - accuracy: 0.1945
Epoch 6/100
 - 0s - loss: 2.9813 - accuracy: 0.1945
Epoch 7/100
 - 0s - loss: 2.9669 - accuracy: 0.1945
Epoch 8/100
 - 0s - loss: 2.9524 - accuracy: 0.1945
Epoch 9/100
 - 0s - loss: 2.9408 - accuracy: 0.1945
Epoch 10/100
 - 0s - loss: 2.9203 - accuracy: 0.1945
Epoch 11/100
 - 0s - loss: 2.9039 - accuracy: 0.1945
Epoch 12/100
 - 0s - loss: 2.8825 - accuracy: 0.1945
Epoch 13/100
 - 0s - loss: 2.8570 - accuracy: 0.1945
Epoch 14/100
 - 0s - loss: 2.8332 - accuracy: 0.1945
Epoch 15/100
 - 0s - loss: 2.8101 - accuracy: 0.1970
Epoch 16/100
 - 0s - loss: 2.7798 - accuracy: 0.2170
Epoch 17/100
 - 0s - loss: 2.7520 - accuracy: 0.2195
Epoch 18/100
 - 0s - loss: 2.7090 - accuracy: 0.2195
Epoch 19/100
 - 0s - loss: 2.6775 - accuracy: 0.2394
Ep

In [24]:
model.save('model.h5')

In [25]:
from pickle import dump
dump(mapping,open('mapping.pkl','wb'))

In [26]:
# Generate text

In [28]:
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [29]:
model = load_model('model.h5')

In [30]:
mapping = load(open('mapping.pkl','rb'))

In [49]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    for _ in range(n_chars):
        encoded = [mapping[char] for char in in_text]
        
        encoded = pad_sequences([encoded], maxlen = seq_length, truncating='pre')
        
        encoded = to_categorical(encoded, num_classes=len(mapping))
        encoded = encoded.reshape(1,encoded.shape[1], encoded.shape[2])
        yhat = model.predict_classes(encoded, verbose=0)
        
        out_char = ''
        for char,index in mapping.items():
            if index == yhat:
                out_char = char
                break
        
        in_text += char
    
    return in_text

In [50]:
print(generate_seq(model, mapping, 10, 'Sing a son', 20))

Sing a song of sixpence, A poc


In [52]:
print(generate_seq(model, mapping, 10, 'king was i', 20))

king was in his counting house


In [53]:
print(generate_seq(model, mapping, 10, 'hello worl', 20))

hello worlse Wha int ppeedolof
